### Calculate variance and variance images for stripe fixation data

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'svg'

Populating the interactive namespace from numpy and matplotlib


In [2]:
import group_data as gd

In [3]:
def calc_and_save_img_stats(fly):
    import numpy as np
    import cPickle
    def wing_correlations_df(sig,imgs,fly = None):
        #calculate some summary images including the correlation image
        include = np.isfinite(sig)
        sig = sig[include]
        imgs = imgs[include]
        #now calculate stats
        mean_img = np.mean(imgs,axis=0)
        zeroed_img = imgs - mean_img

        max_img = np.max(imgs,axis = 0)
        norm_img = np.linalg.norm(zeroed_img,axis = 0)
        norm_sig = np.linalg.norm(sig-np.mean(sig))
        dot_img = np.sum(zeroed_img*(sig[:,np.newaxis,np.newaxis]-np.mean(sig)),axis = 0)
        corr_img = dot_img/(norm_img*norm_sig)
        return {'mean_img':mean_img,
            'max_img':max_img,
            'norm_img':norm_img,
            'dot_img':dot_img,
            'corr_img':corr_img}

    def imgs_stats_df(fly = None):
        #get signals from fly
        expmnt = fly.get_expmnt('step_yaw_mod1')
        rwing = np.array(expmnt['tiff_data']['axon_framebase']['Ph1'])
        lwing = np.array(expmnt['tiff_data']['axon_framebase']['Ph0'])
        wbf = np.array(expmnt['tiff_data']['axon_framebase']['wb_frequency'])
        StimCond = np.array(expmnt['tiff_data']['axon_framebase']['StimCond'])
        images = np.array(expmnt['tiff_data']['images'])
        mask = ((StimCond<0) & (wbf >150))[:np.shape(images)[0]]
        #load baseline data
        fi = open(fly.fly_path + 'epoch_data.cpkl')
        
        baseline_range = cPickle.load(fi)['baseline_F']
        fi.close()
        F = np.mean(images[baseline_range],axis = 0)
        #calculate dF/F
        imgs = (images[mask]-F)/F
        #print sum(mask)
        fly.close_fly()
        return {'lwing':wing_correlations_df(lwing[mask],imgs),
                'rwing':wing_correlations_df(rwing[mask],imgs),
                'freq':wing_correlations_df(wbf[mask],imgs)}

    try:
        stats = imgs_stats_df(fly = fly)
        fi = open(fly.fly_path + 'sf_cov_images.cpkl','wb')
        cPickle.dump(stats,fi)
        fi.close()
    except IOError:
        print fly.fly_num

In [4]:
#get the ipython clients
from IPython.parallel import Client
clients = Client() 
clients.block = True
print clients.ids
v = clients[:]

[0, 1, 2, 3, 4, 5, 6, 7]


In [5]:
import cPickle
for swarm_name,swarm in gd.swarms.items():
       v.map(calc_and_save_img_stats,swarm.flies)

CompositeError: one or more exceptions from call to method: calc_and_save_img_stats
[Engine Exception]EngineError: Engine 0 died while running task '24cc0e07-980a-40ca-8844-5ff4a55a1df4'
[3:apply]: MemoryError: 